In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder,StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train.head()

,portfolio_id,desk_id,office_id,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status,return
0,PF00001002,DSK00001001,OFF00001002,B,20040720,110000000.0,T,0.02074,USD,2.332216,1.098097e+08,20040720,NaN,20040812,B,NaN,NaN,0.02496
1,PF00001003,DSK00001002,OFF00001001,A,20040709,176671000.0,N,0.02074,GBP,5.269617,1.760084e+08,20040723,NaN,20040812,C,NaN,NaN,0.05496
2,PF00001005,DSK00001004,OFF00001001,A,20040723,56474000.0,T,0.02074,USD,2.332216,5.637953e+07,20040723,NaN,20040817,A,NaN,NaN,0.02496
3,PF00001006,DSK00001005,OFF00001001,A,20040609,164813000.0,T,0.02074,USD,2.332216,1.645088e+08,20040723,NaN,20040713,A,NaN,NaN,0.02496
4,PF00001007,DSK00001005,OFF00001002,B,20040609,140800000.0,T,0.02074,USD,2.332216,1.405402e+08,20040723,NaN,20040713,B,NaN,NaN,0.02496


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9366 entries, 0 to 9365
Data columns (total 18 columns):
portfolio_id      9366 non-null object
desk_id           5701 non-null object
office_id         9366 non-null object
pf_category       9366 non-null object
start_date        9366 non-null int64
sold              9364 non-null float64
country_code      9366 non-null object
euribor_rate      9366 non-null float64
currency          9366 non-null object
libor_rate        8892 non-null float64
bought            9364 non-null float64
creation_date     9366 non-null int64
indicator_code    3667 non-null object
sell_date         9366 non-null int64
type              9366 non-null object
hedge_value       3665 non-null object
status            6282 non-null object
return            9366 non-null float64
dtypes: float64(5), int64(3), object(10)
memory usage: 1.3+ MB


In [5]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4801 entries, 0 to 4800
Data columns (total 17 columns):
portfolio_id      4801 non-null object
desk_id           2853 non-null object
office_id         4801 non-null object
pf_category       4801 non-null object
start_date        4801 non-null int64
sold              4801 non-null float64
country_code      4801 non-null object
euribor_rate      4801 non-null float64
currency          4801 non-null object
libor_rate        4536 non-null float64
bought            4801 non-null float64
creation_date     4801 non-null int64
indicator_code    1950 non-null object
sell_date         4801 non-null int64
type              4801 non-null object
hedge_value       1950 non-null object
status            3344 non-null object
dtypes: float64(4), int64(3), object(10)
memory usage: 637.7+ KB


In [6]:
test['return']=np.nan

In [7]:
alldata = pd.concat([train, test]).reset_index(drop=True)

In [8]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14167 entries, 0 to 14166
Data columns (total 18 columns):
portfolio_id      14167 non-null object
desk_id           8554 non-null object
office_id         14167 non-null object
pf_category       14167 non-null object
start_date        14167 non-null int64
sold              14165 non-null float64
country_code      14167 non-null object
euribor_rate      14167 non-null float64
currency          14167 non-null object
libor_rate        13428 non-null float64
bought            14165 non-null float64
creation_date     14167 non-null int64
indicator_code    5617 non-null object
sell_date         14167 non-null int64
type              14167 non-null object
hedge_value       5615 non-null object
status            9626 non-null object
return            9366 non-null float64
dtypes: float64(5), int64(3), object(10)
memory usage: 1.9+ MB


In [9]:
alldata.head(10)

,portfolio_id,desk_id,office_id,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status,return
0,PF00001002,DSK00001001,OFF00001002,B,20040720,110000000.0,T,0.02074,USD,2.332216,1.098097e+08,20040720,NaN,20040812,B,NaN,NaN,0.02496
1,PF00001003,DSK00001002,OFF00001001,A,20040709,176671000.0,N,0.02074,GBP,5.269617,1.760084e+08,20040723,NaN,20040812,C,NaN,NaN,0.05496
2,PF00001005,DSK00001004,OFF00001001,A,20040723,56474000.0,T,0.02074,USD,2.332216,5.637953e+07,20040723,NaN,20040817,A,NaN,NaN,0.02496
3,PF00001006,DSK00001005,OFF00001001,A,20040609,164813000.0,T,0.02074,USD,2.332216,1.645088e+08,20040723,NaN,20040713,A,NaN,NaN,0.02496
4,PF00001007,DSK00001005,OFF00001002,B,20040609,140800000.0,T,0.02074,USD,2.332216,1.405402e+08,20040723,NaN,20040713,B,NaN,NaN,0.02496
5,PF00001008,DSK00001006,OFF00001001,A,20040707,48741000.0,T,0.02074,USD,2.332216,4.865127e+07,20040726,NaN,20040810,A,NaN,NaN,0.02490
6,PF00001010,DSK00001009,OFF00001001,A,20040706,60593500.0,T,0.02074,USD,2.332216,6.048181e+07,20040726,NaN,20040809,A,NaN,NaN,0.02493
7,PF00001011,DSK00001009,OFF00001002,B,20040706,134200000.0,T,0.02074,USD,2.332216,1.339526e+08,20040726,NaN,20040809,B,NaN,NaN,0.02493
8,PF00001012,DSK00001010,OFF00001001,A,20040419,82929000.0,T,0.02074,USD,2.332216,8.250616e+07,20040726,NaN,20040720,A,NaN,NaN,0.02460
9,PF00001016,DSK00001014,OFF00001001,A,20040414,212476000.0,T,0.02074,USD,2.332216,2.114140e+08,20040727,NaN,20040713,A,NaN,NaN,0.02466


In [10]:
def trim(text):
    text=text[-1:-5:-1]
    return int(text)

trim('DSK00001001')

1001

In [11]:
alldata[['country_code' , 'return']].groupby(['country_code'], as_index=False).mean().sort_values(by='return',ascending=False)

,country_code,return
4,Z,0.061187
1,N,0.023418
2,T,0.018381
0,M,0.006641
3,U,-0.001524


In [12]:
alldata['sold'].median()

56045000.0

In [13]:
alldata['sold'].fillna(train['sold'].median(), inplace=True)

In [14]:
alldata['libor_rate'].fillna(5.365030, inplace=True)

In [15]:
alldata['bought'].fillna(train['bought'].median(), inplace=True)

In [16]:
alldata['indicator_code'].fillna(False, inplace=True)

In [17]:
alldata['hedge_value'].fillna(False, inplace=True)

In [18]:
alldata['status'].fillna(False, inplace=True)

In [19]:
alldata = alldata.drop(['desk_id'], axis=1)

In [20]:
alldata['office_id']=alldata['office_id'].map(lambda x:trim(x))

In [21]:
alldata.head()

,portfolio_id,office_id,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,bought,creation_date,indicator_code,sell_date,type,hedge_value,status,return
0,PF00001002,2001,B,20040720,110000000.0,T,0.02074,USD,2.332216,1.098097e+08,20040720,False,20040812,B,False,False,0.02496
1,PF00001003,1001,A,20040709,176671000.0,N,0.02074,GBP,5.269617,1.760084e+08,20040723,False,20040812,C,False,False,0.05496
2,PF00001005,1001,A,20040723,56474000.0,T,0.02074,USD,2.332216,5.637953e+07,20040723,False,20040817,A,False,False,0.02496
3,PF00001006,1001,A,20040609,164813000.0,T,0.02074,USD,2.332216,1.645088e+08,20040723,False,20040713,A,False,False,0.02496
4,PF00001007,2001,B,20040609,140800000.0,T,0.02074,USD,2.332216,1.405402e+08,20040723,False,20040713,B,False,False,0.02496


In [22]:
alldata.columns

Index(['portfolio_id', 'office_id', 'pf_category', 'start_date', 'sold',
       'country_code', 'euribor_rate', 'currency', 'libor_rate', 'bought',
       'creation_date', 'indicator_code', 'sell_date', 'type', 'hedge_value',
       'status', 'return'],
      dtype='object')

In [23]:
cols=['pf_category','country_code', 'currency','indicator_code', 'type', 'hedge_value','status']

for x in cols:
    lbl=LabelEncoder()
    alldata[x]=lbl.fit_transform(alldata[x])

In [24]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14167 entries, 0 to 14166
Data columns (total 17 columns):
portfolio_id      14167 non-null object
office_id         14167 non-null int64
pf_category       14167 non-null int64
start_date        14167 non-null int64
sold              14167 non-null float64
country_code      14167 non-null int64
euribor_rate      14167 non-null float64
currency          14167 non-null int64
libor_rate        14167 non-null float64
bought            14167 non-null float64
creation_date     14167 non-null int64
indicator_code    14167 non-null int64
sell_date         14167 non-null int64
type              14167 non-null int64
hedge_value       14167 non-null int64
status            14167 non-null int64
return            9366 non-null float64
dtypes: float64(5), int64(11), object(1)
memory usage: 1.8+ MB


In [25]:
train=alldata[:len(train)]
test=alldata[len(train):]

In [26]:
ids = test['portfolio_id']
test = test.drop(['portfolio_id','return'], axis=1)
y = train['return']
train = train.drop(['portfolio_id', 'return'], axis=1)

In [27]:
train.shape, test.shape

((9366, 15), (4801, 15))

# MODEL

In [28]:
forest_reg = RandomForestRegressor(random_state=7)
scores = cross_val_score(forest_reg, train, y, scoring='r2', cv=5)
print(scores)
print('mean r2:',np.mean(scores))

[-0.03983869  0.97657537  0.93351925  0.39467592  0.74896837]
mean r2: 0.60278004448


In [29]:
forest_reg = RandomForestRegressor(random_state=7)
forest_reg.fit(train, y)
preds = forest_reg.predict(test)

sub = pd.DataFrame({'portfolio_id': ids, 'return': preds})
filename = 'sub_returns.csv'
sub.to_csv(filename, index=False)

Linear Regression

In [30]:
lin_reg=LinearRegression(normalize=True)
lin_reg.fit(train,y)
pred_lin=lin_reg.predict(test)

sub_linear = pd.DataFrame({'portfolio_id': ids, 'return': pred_lin})
filename2 = 'sub_lin_returns.csv'
sub_linear.to_csv(filename2, index=False)

Random Forest is giving score of 0.94764 
while
Linear Regression of 0.87374